# Instalación del AWS SDK para Python

Primero que nada, instalamos boto3, el SDK para Python de AWS

In [ ]:
import os

os.system(f"pip install boto3")

# Conectamos con dynamo

Luego conectamos con la instancia de dynamo que querramos usar

In [ ]:
from __future__ import print_function # Python 2/3 compatibility
import boto3, pprint

dynamodb = boto3.resource('dynamodb', endpoint_url="http://172.21.0.5:8000", 
                          aws_access_key_id="<aws_access_key_id>", aws_secret_access_key= "<aws_secret_access_key>", 
                          region_name='us-west-2')
pprint.pprint(dynamodb)

# Tablas

Verificamos si existen tablas actualmente

In [ ]:
response = dynamodb.meta.client.list_tables()

pprint.pprint(response)

# Creación de Tablas

Creamos una tabla sencilla, con los parámetros básicos

In [ ]:
table = dynamodb.create_table(
    TableName="Movies", 
    BillingMode="PROVISIONED",
    KeySchema=[
        {'AttributeName': 'Year', 'KeyType': 'HASH'},  # Partition key
        {'AttributeName': 'Title', 'KeyType': 'RANGE'}  # Sort key
    ],
    AttributeDefinitions=[
        {'AttributeName': 'Year', 'AttributeType': 'S'},
        {'AttributeName': 'Title', 'AttributeType': 'S'}
    ],
    ProvisionedThroughput={'ReadCapacityUnits': 10, 'WriteCapacityUnits': 10})

table.meta.client.get_waiter('table_exists').wait(TableName='Movies')
print('Table created, please continue to insert data.')

# Inserción

Agregamos un elemento a la tabla

In [ ]:
table.put_item(
    Item={
    "Title": "Avatar",
    "Year": "2009",
    "Rated": "PG-13",
    "Released": "18 Dec 2009",
    "Runtime": "162 min",
    "Genre": "Action, Adventure, Fantasy",
    "Director": "James Cameron",
    "Writer": "James Cameron",
    "Actors": "Sam Worthington, Zoe Saldana, Sigourney Weaver, Stephen Lang",
    "Plot": "A paraplegic marine dispatched to the moon Pandora on a unique mission becomes torn between following his orders and protecting the world he feels is his home.",
    "Language": "English, Spanish",
    "Country": "USA, UK",
    "Awards": "Won 3 Oscars. Another 80 wins & 121 nominations.",
    "Poster": "http://ia.media-imdb.com/images/M/MV5BMTYwOTEwNjAzMl5BMl5BanBnXkFtZTcwODc5MTUwMw@@._V1_SX300.jpg",
    "Metascore": "83",
    "imdbRating": "7.9",
    "imdbVotes": "890,617",
    "imdbID": "tt0499549",
    "Type": "movie",
    "Response": "True",
    "Images": [
      "https://images-na.ssl-images-amazon.com/images/M/MV5BMjEyOTYyMzUxNl5BMl5BanBnXkFtZTcwNTg0MTUzNA@@._V1_SX1500_CR0,0,1500,999_AL_.jpg",
      "https://images-na.ssl-images-amazon.com/images/M/MV5BNzM2MDk3MTcyMV5BMl5BanBnXkFtZTcwNjg0MTUzNA@@._V1_SX1777_CR0,0,1777,999_AL_.jpg",
      "https://images-na.ssl-images-amazon.com/images/M/MV5BMTY2ODQ3NjMyMl5BMl5BanBnXkFtZTcwODg0MTUzNA@@._V1_SX1777_CR0,0,1777,999_AL_.jpg",
      "https://images-na.ssl-images-amazon.com/images/M/MV5BMTMxOTEwNDcxN15BMl5BanBnXkFtZTcwOTg0MTUzNA@@._V1_SX1777_CR0,0,1777,999_AL_.jpg",
      "https://images-na.ssl-images-amazon.com/images/M/MV5BMTYxMDg1Nzk1MV5BMl5BanBnXkFtZTcwMDk0MTUzNA@@._V1_SX1500_CR0,0,1500,999_AL_.jpg"
    ]
  })

Agregamos varios mas traidos de un archivo de ejemplo

In [ ]:
from decimal import Decimal
import json

with open('../datasets/movies.json') as json_file:
    data = json.load(json_file, parse_float=Decimal)
    for movie in data:
        table.put_item(Item=movie)

#pprint.pprint(data)
pprint.pprint(len(data))

# Obtenemos un elemento

Utilizando la clave primaria completa, obtenemos un item

In [ ]:
response = table.get_item(Key={'Year': '2009', 'Title': "Avatar"})
pprint.pprint(response['Item'])

# Consulta

Realizamos una query para traer todos los items de un año en particular

In [ ]:
from boto3.dynamodb.conditions import Key

response = table.query(KeyConditionExpression=Key('Year').eq('2016'))
pprint.pprint(response['Items'])

# Actualización

Modificamos un elemento existente

In [ ]:
response = table.update_item(
                Key={'Year': '2009', 'Title': "Avatar"},
                UpdateExpression="set imdbRating=:i, Runtime=:r",
                ExpressionAttributeValues={
                    ':i': "8.1", ':r': "163 min"},
                ReturnValues="UPDATED_NEW")
pprint.pprint(response)

# Actualización

Agregamos un elemento a una lista existente

In [ ]:
response = table.update_item(
                Key={'Year': '2009', 'Title': "Avatar"},
                UpdateExpression="set Images = list_append(Images, :i)",
                ExpressionAttributeValues={
                    ':i': ["http://ia.media-imdb.com/images/M/MV5BMTYwOTEwNjAzMl5BMl5BanBnXkFtZTcwODc5MTUwMw@@._V1_SX300.jpg"]},
                ReturnValues="UPDATED_NEW")
pprint.pprint(response)

# Scan

Realizamos un scan para obtener todos los elementos de la tabla

In [ ]:
response = table.scan()
print("Items Returned: {}".format(len(response['Items'])))
#pprint.pprint(response)

# Borramos la tabla

Finalmente, eliminamos la tabla

In [ ]:
table.delete()

print(f"Deleting {table.name}...")
table.wait_until_not_exists()

print('Table deleted.')